# Analisis de superficial del csv de clicks
### En este notebook exploramos los registros del csv con el objetivo de buscar relaciones entre las distintas variables

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Leemos el csv, y dropeamos las columnas action_id por estar llena de nans y wifi_connection por estar llena de False

In [2]:
clicks = pd.read_csv("../data/clicks.csv", dtype={'advertiser_id': 'category', 'action_id': 'float64', 'source_id': 'category',
                                               'created': 'str', 'country_code':'category', 'latitude': 'float64',
                                               'longitude':'float64', 'wifi_connection':'bool' , 'carrier_id':np.float16,
                                               'trans_id':'str', 'os_minor':'float64', 'agent_device':'float64', 'os_major':'float64',
                                               'specs_brand':'int64', 'brand':'category', 'timeToClick':'float64',
                                               'ref_type':'category', 'ref_hash':'int64'}, parse_dates = ['created'])
clicks.drop('action_id', axis=1, inplace = True)
clicks.drop('wifi_connection', axis=1, inplace = True) #false
clicks['day'] = clicks['created'].dt.date
clicks['hour'] = clicks['created'].dt.hour
clicks.head(5)

/home/gonzalo/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,advertiser_id,source_id,created,country_code,latitude,longitude,carrier_id,trans_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_type,ref_hash,day,hour
0,1,2,2019-04-18 05:27:42.197000+00:00,6287817205707153877,1.714547,0.871535,3.0,9JMAfrb-b9cSEVCJb0P9JfihGthaS7E,1.517644e+18,NaN,5.131616e+18,71913840936116953,0.0,2.317,0.968,0.503,1891515180541284343,1293710398598742392,2019-04-18,5
1,1,1,2019-04-18 05:27:03.164000+00:00,6287817205707153877,1.714512,0.871062,2.0,r3xtTRv2lInfiXG8JI3NQsNcBo8GyFQ,1.288578e+18,NaN,3.908390e+18,3576558787748411622,1.0,7.653,0.712,1.689,1891515180541284343,1663930990551616564,2019-04-18,5
2,1,1,2019-04-18 05:42:07.926000+00:00,6287817205707153877,1.714547,0.871535,4.0,WOnHFqQtY48z_ygKZ-030U_g0TMGVMw,2.238736e+18,NaN,3.581233e+18,3576558787748411622,NaN,464.796,0.227,0.251,1891515180541284343,8488038938665586188,2019-04-18,5
3,1,1,2019-04-18 05:26:04.446000+00:00,6287817205707153877,1.708041,0.870772,1.0,wQMLLmYqiFhSuha9p9B13PMtcyBW_vM,2.411640e+18,NaN,3.908390e+18,3576558787748411622,NaN,225.311,0.696,6.587,1891515180541284343,6488361690105189959,2019-04-18,5
4,1,1,2019-04-18 05:23:37.764000+00:00,6287817205707153877,1.715514,0.870772,2.0,GeFoyBzMA7taylMxxjzlNPTU-n4FXFs,1.517644e+18,NaN,5.131616e+18,3576558787748411622,0.0,84.736,0.059,0.142,1891515180541284343,1348993302102753419,2019-04-18,5


In [3]:
clicks['created'].max()

Timestamp('2019-04-26 23:59:22.065000+0000', tz='UTC')

In [4]:
clicks['created'].min()

Timestamp('2019-04-12 00:00:01.981000+0000', tz='UTC')

In [5]:
clicks.dtypes

advertiser_id               category
source_id                   category
created          datetime64[ns, UTC]
country_code                category
latitude                     float64
longitude                    float64
carrier_id                   float16
trans_id                      object
os_minor                     float64
agent_device                 float64
os_major                     float64
specs_brand                    int64
brand                       category
timeToClick                  float64
touchX                        object
touchY                        object
ref_type                    category
ref_hash                       int64
day                           object
hour                           int64
dtype: object

## Realizamos un histograma del TimeToClick

In [6]:
clicks_acotado = clicks.dropna(subset= ['timeToClick'])

## Creamos una columna categorica del TimeToClick

In [7]:
bins = pd.IntervalIndex.from_tuples([(600, 120000), (180, 600), (60, 180), (15, 60), (3, 15), (0, 3)])
clicks_acotado['time_categ'] = pd.cut(clicks_acotado['timeToClick'],bins)
clicks_acotado['time_categ'].cat.categories = ['10 a 2000min','3 a 10min','1 a 3 min','15 a 60seg','3 a 15seg','0 a 3 seg']

clicks['time_categ'] = pd.cut(clicks['timeToClick'],bins)
clicks['time_categ'].cat.categories = ['10 a 2000min','3 a 10min','1 a 3 min','15 a 60seg', '3 a 15seg','0 a 3 seg']

clicks_acotado['time_categ'].value_counts()

/home/gonzalo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0 a 3 seg       12856
3 a 15seg        9285
15 a 60seg       6707
1 a 3 min        3992
3 a 10min        2907
10 a 2000min     2431
Name: time_categ, dtype: int64

## Elegimos las columnas con menor entropia o pocos valores diferentes y nos quedamos con los 5 valores mas comunes

In [8]:
grupos = ['advertiser_id', 'carrier_id', 'os_minor', 'agent_device', 'os_major', 'specs_brand', 'ref_type', 'hour', 'day', 'time_categ']
top5 = {}
for i in grupos:
    top5[i]= clicks[i].value_counts().head(5).index.tolist()
top5

{'advertiser_id': ['2', '1', '0', '3', '4'],
 'agent_device': [6.794880020077884e+18,
  9.17380693425856e+18,
  9.186120447236368e+18,
  1.9540924917876943e+18,
  2.8270958855039072e+17],
 'carrier_id': [1.0, 7.0, 2.0, 0.0, 13.0],
 'day': [datetime.date(2019, 4, 26),
  datetime.date(2019, 4, 25),
  datetime.date(2019, 4, 23),
  datetime.date(2019, 4, 24),
  datetime.date(2019, 4, 22)],
 'hour': [3, 4, 2, 1, 16],
 'os_major': [5.131615556736863e+18,
  3.9083902007568794e+18,
  5.754947116114108e+18,
  5.648867414868049e+18,
  3.5812325749809167e+18],
 'os_minor': [1.5176438893491397e+18,
  6.795761880764845e+18,
  3.5759630297247805e+18,
  1.2885781261232225e+18,
  7.531669329342817e+18],
 'ref_type': ['1891515180541284343', '1494519392962156891'],
 'specs_brand': [71913840936116953,
  3576558787748411622,
  6341583823913642480,
  784329784168794382,
  4222063286888578800],
 'time_categ': ['0 a 3 seg',
  '3 a 15seg',
  '15 a 60seg',
  '1 a 3 min',
  '3 a 10min']}

In [9]:
clicks = clicks.sort_values(by = 'created')
clicks = clicks.sort_values(by = 'ref_hash')
clicks['repeticiones'] = 1
device_ids = clicks['ref_hash'].to_frame().set_index('ref_hash')
#clicks = pd.get_dummies(clicks, columns = grupos)
for column in grupos:
    for value in top5[column]:
        clicks[column + str(value)] = (clicks[column] == value)*1

In [10]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-21 00:00:00', utc = True)
clicks_ventana1=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [11]:
fecha_minima=pd.to_datetime('2019-04-19 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-22 00:00:00', utc = True)
clicks_ventana2=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [12]:
fecha_minima=pd.to_datetime('2019-04-20 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-23 00:00:00', utc = True)
clicks_ventana3=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [13]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-24 00:00:00', utc = True)
clicks_ventana4=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [14]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-27 00:00:00', utc = True)
clicks_ventana7=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [17]:
features = ['sum', 'mean', 'std', 'min', 'max']
ventanas = [clicks_ventana1, clicks_ventana2, clicks_ventana3, clicks_ventana4, clicks_ventana7]
numero_de_ventanas = [1,2,3,4,7]
drop_columns_st = [
    'latitude_sum',
    'latitude_mean',
    'latitude_std',
    'latitude_min',
    'latitude_max',
    'longitude_sum',
    'longitude_mean',
    'longitude_std',
    'longitude_min',
    'longitude_max',
    'carrier_id_sum',
    'carrier_id_mean',
    'carrier_id_std',
    'carrier_id_min',
    'carrier_id_max',
    'os_minor_sum',
    'os_minor_mean',
    'os_minor_std',
    'os_minor_min',
    'os_minor_max',
    'agent_device_sum',
    'agent_device_mean',
    'agent_device_std',
    'agent_device_min',
    'agent_device_max',
    'os_major_sum',
    'os_major_mean',
    'os_major_std',
    'os_major_min',
    'os_major_max',
    'specs_brand_sum',
    'specs_brand_mean',
    'specs_brand_std',
    'specs_brand_min',
    'specs_brand_max',
    'timeToClick_sum',
    'timeToClick_mean',
    'timeToClick_std',
    'timeToClick_min',
    'timeToClick_max',
    'hour_sum',
    'hour_mean',
    'hour_std',
    'hour_min',
    'hour_max',
    'repeticiones_sum',
    'repeticiones_mean',
    'repeticiones_std',
    'repeticiones_min',
    'repeticiones_max',
    'advertiser_id2_sum',
    'advertiser_id2_mean',
    'advertiser_id2_std',
    'advertiser_id2_min',
    'advertiser_id2_max',
    'advertiser_id1_sum',
    'advertiser_id1_mean',
    'advertiser_id1_std',
    'advertiser_id1_min',
    'advertiser_id1_max',
    'advertiser_id0_sum',
    'advertiser_id0_mean',
    'advertiser_id0_std',
    'advertiser_id0_min',
    'advertiser_id0_max',
    'advertiser_id3_sum',
    'advertiser_id3_mean',
    'advertiser_id3_std',
    'advertiser_id3_min',
    'advertiser_id3_max',
    'advertiser_id4_sum',
    'advertiser_id4_mean',
    'advertiser_id4_std',
    'advertiser_id4_min',
    'advertiser_id4_max',
    'carrier_id1.0_sum',
    'carrier_id1.0_mean',
    'carrier_id1.0_std',
    'carrier_id1.0_min',
    'carrier_id1.0_max',
    'carrier_id7.0_sum',
    'carrier_id7.0_mean',
    'carrier_id7.0_std',
    'carrier_id7.0_min',
    'carrier_id7.0_max',
    'carrier_id2.0_sum',
    'carrier_id2.0_mean',
    'carrier_id2.0_std',
    'carrier_id2.0_min',
    'carrier_id2.0_max',
    'carrier_id0.0_sum',
    'carrier_id0.0_mean',
    'carrier_id0.0_std',
    'carrier_id0.0_min',
    'carrier_id0.0_max',
    'carrier_id13.0_sum',
    'carrier_id13.0_mean',
    'carrier_id13.0_std',
    'carrier_id13.0_min',
    'carrier_id13.0_max',
    'os_minor1.51764388935e+18_sum',
    'os_minor1.51764388935e+18_mean',
    'os_minor1.51764388935e+18_std',
    'os_minor1.51764388935e+18_min',
    'os_minor1.51764388935e+18_max',
    'os_minor6.79576188076e+18_sum',
    'os_minor6.79576188076e+18_mean',
    'os_minor6.79576188076e+18_std',
    'os_minor6.79576188076e+18_min',
    'os_minor6.79576188076e+18_max',
    'os_minor3.57596302972e+18_sum',
    'os_minor3.57596302972e+18_mean',
    'os_minor3.57596302972e+18_std',
    'os_minor3.57596302972e+18_min',
    'os_minor3.57596302972e+18_max',
    'os_minor1.28857812612e+18_sum',
    'os_minor1.28857812612e+18_mean',
    'os_minor1.28857812612e+18_std',
    'os_minor1.28857812612e+18_min',
    'os_minor1.28857812612e+18_max',
    'os_minor7.53166932934e+18_sum',
    'os_minor7.53166932934e+18_mean',
    'os_minor7.53166932934e+18_std',
    'os_minor7.53166932934e+18_min',
    'os_minor7.53166932934e+18_max',
    'agent_device6.79488002008e+18_sum',
    'agent_device6.79488002008e+18_mean',
    'agent_device6.79488002008e+18_std',
    'agent_device6.79488002008e+18_min',
    'agent_device6.79488002008e+18_max',
    'agent_device9.17380693426e+18_sum',
    'agent_device9.17380693426e+18_mean',
    'agent_device9.17380693426e+18_std',
    'agent_device9.17380693426e+18_min',
    'agent_device9.17380693426e+18_max',
    'agent_device9.18612044724e+18_sum',
    'agent_device9.18612044724e+18_mean',
    'agent_device9.18612044724e+18_std',
    'agent_device9.18612044724e+18_min',
    'agent_device9.18612044724e+18_max',
    'agent_device1.95409249179e+18_sum',
    'agent_device1.95409249179e+18_mean',
    'agent_device1.95409249179e+18_std',
    'agent_device1.95409249179e+18_min',
    'agent_device1.95409249179e+18_max',
    'agent_device2.8270958855e+17_sum',
    'agent_device2.8270958855e+17_mean',
    'agent_device2.8270958855e+17_std',
    'agent_device2.8270958855e+17_min',
    'agent_device2.8270958855e+17_max',
    'os_major5.13161555674e+18_sum',
    'os_major5.13161555674e+18_mean',
    'os_major5.13161555674e+18_std',
    'os_major5.13161555674e+18_min',
    'os_major5.13161555674e+18_max',
    'os_major3.90839020076e+18_sum',
    'os_major3.90839020076e+18_mean',
    'os_major3.90839020076e+18_std',
    'os_major3.90839020076e+18_min',
    'os_major3.90839020076e+18_max',
    'os_major5.75494711611e+18_sum',
    'os_major5.75494711611e+18_mean',
    'os_major5.75494711611e+18_std',
    'os_major5.75494711611e+18_min',
    'os_major5.75494711611e+18_max',
    'os_major5.64886741487e+18_sum',
    'os_major5.64886741487e+18_mean',
    'os_major5.64886741487e+18_std',
    'os_major5.64886741487e+18_min',
    'os_major5.64886741487e+18_max',
    'os_major3.58123257498e+18_sum',
    'os_major3.58123257498e+18_mean',
    'os_major3.58123257498e+18_std',
    'os_major3.58123257498e+18_min',
    'os_major3.58123257498e+18_max',
    'specs_brand71913840936116953_sum',
    'specs_brand71913840936116953_mean',
    'specs_brand71913840936116953_std',
    'specs_brand71913840936116953_min',
    'specs_brand71913840936116953_max',
    'specs_brand3576558787748411622_sum',
    'specs_brand3576558787748411622_mean',
    'specs_brand3576558787748411622_std',
    'specs_brand3576558787748411622_min',
    'specs_brand3576558787748411622_max',
    'specs_brand6341583823913642480_sum',
    'specs_brand6341583823913642480_mean',
    'specs_brand6341583823913642480_std',
    'specs_brand6341583823913642480_min',
    'specs_brand6341583823913642480_max',
    'specs_brand784329784168794382_sum',
    'specs_brand784329784168794382_mean',
    'specs_brand784329784168794382_std',
    'specs_brand784329784168794382_min',
    'specs_brand784329784168794382_max',
    'specs_brand4222063286888578800_sum',
    'specs_brand4222063286888578800_mean',
    'specs_brand4222063286888578800_std',
    'specs_brand4222063286888578800_min',
    'specs_brand4222063286888578800_max',
    'ref_type1891515180541284343_sum',
    'ref_type1891515180541284343_mean',
    'ref_type1891515180541284343_std',
    'ref_type1891515180541284343_min',
    'ref_type1891515180541284343_max',
    'ref_type1494519392962156891_sum',
    'ref_type1494519392962156891_mean',
    'ref_type1494519392962156891_std',
    'ref_type1494519392962156891_min',
    'ref_type1494519392962156891_max',
    'hour3_sum',
    'hour3_mean',
    'hour3_std',
    'hour3_min',
    'hour3_max',
    'hour4_sum',
    'hour4_mean',
    'hour4_std',
    'hour4_min',
    'hour4_max',
    'hour2_sum',
    'hour2_mean',
    'hour2_std',
    'hour2_min',
    'hour2_max',
    'hour1_sum',
    'hour1_mean',
    'hour1_std',
    'hour1_min',
    'hour1_max',
    'hour16_sum',
    'hour16_mean',
    'hour16_std',
    'hour16_min',
    'hour16_max',
    'day2019-04-26_sum',
    'day2019-04-26_mean',
    'day2019-04-26_std',
    'day2019-04-26_min',
    'day2019-04-26_max',
    'day2019-04-25_sum',
    'day2019-04-25_mean',
    'day2019-04-25_std',
    'day2019-04-25_min',
    'day2019-04-25_max',
    'day2019-04-23_sum',
    'day2019-04-23_mean',
    'day2019-04-23_std',
    'day2019-04-23_min',
    'day2019-04-23_max',
    'day2019-04-24_sum',
    'day2019-04-24_mean',
    'day2019-04-24_std',
    'day2019-04-24_min',
    'day2019-04-24_max',
    'day2019-04-22_sum',
    'day2019-04-22_mean',
    'day2019-04-22_std',
    'day2019-04-22_min',
    'day2019-04-22_max',
    'time_categ0 a 3 seg_sum',
    'time_categ0 a 3 seg_mean',
    'time_categ0 a 3 seg_std',
    'time_categ0 a 3 seg_min',
    'time_categ0 a 3 seg_max',
    'time_categ3 a 15seg_sum',
    'time_categ3 a 15seg_mean',
    'time_categ3 a 15seg_std',
    'time_categ3 a 15seg_min',
    'time_categ3 a 15seg_max',
    'time_categ15 a 60seg_sum',
    'time_categ15 a 60seg_mean',
    'time_categ15 a 60seg_std',
    'time_categ15 a 60seg_min',
    'time_categ15 a 60seg_max',
    'time_categ1 a 3 min_sum',
    'time_categ1 a 3 min_mean',
    'time_categ1 a 3 min_std',
    'time_categ1 a 3 min_min',
    'time_categ1 a 3 min_max',
    'time_categ3 a 10min_sum',
    'time_categ3 a 10min_mean',
    'time_categ3 a 10min_std',
    'time_categ3 a 10min_min',
    'time_categ3 a 10min_max']


In [18]:
contador = 0
for ventana in ventanas:
    features_ventana = ventana.groupby('ref_hash').agg(features).fillna(0)
    level0 = features_ventana.columns.get_level_values(0)
    level1 = features_ventana.columns.get_level_values(1)
    features_ventana.columns = level0 + "_" + level1
    features_ventana.drop(drop_columns_st, axis=1, inplace = True)
    features_ventana.to_csv('../xgb/features_clicks_gonzalo_st_ventana' + str(numero_de_ventanas[contador]) + '.csv')
    contador = contador + 1